### Importing required Libraries


In [22]:
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as k

### Getting, Preparing and Preprocessing the Data

In [23]:
batch_size = 128 # The process of splitting the training dataset in n batches ( mini-batches)
num_classes = 10 # initializing parameters
epochs=12 #Epoch : one forward pass and one backward pass of all the training examples 
img_rows, img_cols = 28, 28  # the dimensions of one image (grey scale)
(x_train, y_train),(x_test, y_test) = mnist.load_data() # shape and load the dataset

In [24]:
# define data formats between differet backends
if k.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)


#Normalization of data dimensions
x_train = x_train/255.0
x_test = x_test/255.0
print('x_train shape : ', x_train.shape)
print('x_test shape : ', x_test.shape)

# Convert class vector = labels to binary class matrices because the classification process is produced in binary time as we have 10 labels 
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape :  (60000, 28, 28, 1)
x_test shape :  (10000, 28, 28, 1)


### Create Model : Outlining the model

In [25]:
#Define the model
model= Sequential() #initializing model
model.add(Conv2D(32, kernel_size=(3,3), activation='relu', input_shape=input_shape)) # the first layer is a convolutional layer with 32 filters
model.add(Conv2D(64,(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2))) #Max Pooling #division
model.add(Flatten()) #Flatten Layer is so important # to remove dimensions
model.add(Dense(128, activation='relu')) #Fully connected layer with 128 neurons 
model.add(Dense(num_classes, activation='softmax')) # a layer related to binary classification 
model.summary()



Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_9 (Conv2D)           (None, 26, 26, 32)        320       
                                                                 
 conv2d_10 (Conv2D)          (None, 24, 24, 64)        18496     
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 12, 12, 64)       0         
 2D)                                                             
                                                                 
 flatten_1 (Flatten)         (None, 9216)              0         
                                                                 
 dense_2 (Dense)             (None, 128)               1179776   
                                                                 
 dense_3 (Dense)             (None, 10)                1290      
                                                      

### Compile Model 

In [26]:
model.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adadelta(), metrics=['accuracy'])


### Train Model

In [27]:
model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_data=(x_test, y_test))


Epoch 1/12
469/469 [==============================] - 4s 8ms/step - loss: 2.2718 - accuracy: 0.2026 - val_loss: 2.2161 - val_accuracy: 0.3611
Epoch 2/12
469/469 [==============================] - 4s 9ms/step - loss: 2.1542 - accuracy: 0.4928 - val_loss: 2.0718 - val_accuracy: 0.6139
Epoch 3/12
469/469 [==============================] - 5s 10ms/step - loss: 1.9733 - accuracy: 0.6652 - val_loss: 1.8377 - val_accuracy: 0.7238
Epoch 4/12
469/469 [==============================] - 5s 11ms/step - loss: 1.6871 - accuracy: 0.7314 - val_loss: 1.4885 - val_accuracy: 0.7627
Epoch 5/12
469/469 [==============================] - 5s 10ms/step - loss: 1.3208 - accuracy: 0.7655 - val_loss: 1.1170 - val_accuracy: 0.7948
Epoch 6/12
469/469 [==============================] - 4s 8ms/step - loss: 1.0010 - accuracy: 0.7957 - val_loss: 0.8508 - val_accuracy: 0.8217
Epoch 7/12
469/469 [==============================] - 4s 8ms/step - loss: 0.7926 - accuracy: 0.8183 - val_loss: 0.6905 - val_accuracy: 0.8405
Epo

### Evaluate and test Model

In [28]:
# Evaluate the model on the test set
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.43061724305152893
Test accuracy: 0.881600022315979


### Save Model

In [29]:
model.save("model.h5")
print('saved')

saved


### Restoring Model and Making predictions 

In [36]:
from tensorflow.keras.preprocessing.image import load_img as keras_load_img
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model

# load an image to make a prediction on

def load_image(filename): # a function dedicated to preprocess the img after loading
  img = keras_load_img(filename, target_size=(28, 28), color_mode="grayscale")
  img = img_to_array(img) # convert the image to a array
  img = img.reshape(1,28,28,1) #reshaping the img to single sample with one channel 
  img = img.astype("float32")
  img = img/255.0
  return img

def run_example():
  #loading_img
  img = load_image("/content/sample_image.png")
  # loading the trained model
  model = load_model("/content/model.h5")
  # Predict the class
  result = model.predict(img) # make a prediction using the trained model
  predicted_class = result.argmax(axis=-1)[0]
  print('Predicted class:', predicted_class) # print the predicted class label 

#entry point, run the example
run_example()



#Note that we loaded a pullover image and the index of this label in the mnist dataset is actually 2. 
# the result proves that our prediction is correct


1/1 [==============================] - 0s 132ms/step
Predicted class: 2


### Improve Model's Predictive Accuracy

In [37]:
# We are going to change the parameters of our model by changing some parameters: 
# dense 128 ==> 100 , change the optimizer and number of epochs 

from keras.datasets import fashion_mnist
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import SGD

(trainX, trainY),(testX, testY) = fashion_mnist.load_data() # load the dataset
# reshape dataset to have a single channel 

trainX = trainX.reshape((trainX.shape[0], 28,28,1))
testX = testX.reshape((testX.shape[0], 28,28,1))
#one hot encode target values
trainY = to_categorical(trainY)
testY = to_categorical(testY)
#scale pixels and convert from integers to floats
trainX=trainX.astype("float32")
testX = testX.astype("float32")
trainX=trainX / 255.0
testX = testX / 255.0
#define CNN model
model= Sequential() #initializing model
model.add(Conv2D(32, (3,3), activation='relu', kernel_initializer ="he_uniform", input_shape=(28,28,1))) # the first layer is a convolutional layer with 32 filters
model.add(MaxPooling2D((2,2))) #Max Pooling #division # feature Learning 
model.add(Flatten()) #Flatten Layer is so important # to remove dimensions
model.add(Dense(100, activation='relu', kernel_initializer ="he_uniform")) #Fully connected layer with 128 neurons 
model.add(Dense(10, activation='softmax')) # a layer related to binary classification 
# compile model
opt = SGD(lr=0.01,momentum=0.9)
model.compile(optimizer = opt, loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(trainX, trainY, batch_size=32, epochs=10, verbose=0)
model.save("final_model.h5")


4422102/4422102 [==============================] - 0s 0us/step


/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/gradient_descent.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [39]:
## Evaluate the new model on the test set
score = model.evaluate(testX, testY, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])
# we note that our model is improved based on hyper parameter tuning 

Test loss: 0.30592867732048035
Test accuracy: 0.9016000032424927
